In [12]:
from random import sample
import subprocess

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import subprocess
from pathlib import Path
from audio_clicker_helper import *

In [27]:
#"/usr/users/vogg/sfb1528s3/B06/2023april-july/NewBoxesClosed/"

path_to_files =  "/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/"
cameras = 'all' #'all' #'all' # #'all' or list of cameras [4,7] (also mention camera 7) 
tStart = 100
tEnd2 = 200 # first 60 seconds of second video 200
tEnd = 280 # up to second 100 of main video 280
#which_file = 6 #find out in the cell below which files exist

In [14]:
cam2list = {1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
rootdir = Path(path_to_files)
list_dirs = []


print(os.listdir(rootdir))
for path in os.listdir(rootdir):
    if not path.startswith(".") and not path.startswith("Converted"):
        for subpath in os.listdir(rootdir / path):
            if not subpath.startswith("."):
                list_dirs.append(rootdir / path / subpath)
 
list_dirs.sort()

for i in range(len(list_dirs)):
    print(i, list_dirs[i])

['B', 'Converted', 'J', 'R1', 'X']
0 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-05
1 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-07
2 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-09
3 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-16
4 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-19
5 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/B/2024-04-23
6 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-03
7 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-09
8 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-16
9 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-20
10 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-21
11 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/J/2024-04-23
12 /usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/

In [15]:
which_file = 20
experiment_number = 9
input_path = list_dirs[which_file]
group = input_path.parent.name
print(group)
group_letter = group[:1]
print(group_letter)

output_path = rootdir / "Converted" / group
output_path.mkdir(parents=True, exist_ok=True)

# from https://github.com/scuc/GoPro-Concat-Automation/blob/master/gopro_concat.py


gopr_source_list = []
gopr_key_list = []
gopr_dict = {}

file_list = sorted(os.listdir(input_path))
file_list = [f for f in file_list if f.endswith(".MP4") and not f.startswith(".") and (not "extra" in f)]
print(file_list)


R1
R
['GX010172_cam4.MP4', 'GX010178_cam2.MP4', 'GX010178_cam7.MP4', 'GX010183_cam6.MP4', 'GX010183_cam9.MP4', 'GX010196_cam3.MP4', 'GX010198_cam8.MP4', 'GX010211_cam1.MP4', 'GX020172_cam4.MP4', 'GX020178_cam2.MP4', 'GX020178_cam7.MP4', 'GX020183_cam6.MP4', 'GX020183_cam9.MP4', 'GX020196_cam3.MP4', 'GX020198_cam8.MP4', 'GX020211_cam1.MP4', 'GX030172_cam4.MP4', 'GX030178_cam2.MP4', 'GX030178_cam7.MP4', 'GX030183_cam6.MP4', 'GX030183_cam9.MP4', 'GX030196_cam3.MP4', 'GX030198_cam8.MP4', 'GX030211_cam1.MP4', 'GX040178_cam2.MP4', 'GX040183_cam6.MP4', 'GX040183_cam9.MP4', 'GX040196_cam3.MP4', 'GX040198_cam8.MP4', 'GX040211_cam1.MP4']


In [16]:
# Which GoPro files belong together
# file: GX010046_cam6.MP4 has to be stiched with GX020046_cam6.MP4---file: B_e_1_220918_c1_p1_2.MP4
for file in file_list:
    print("FILE: " + file)
    regex = r"(GX01)" #GX01 always exists, we use it as a key
    gp = re.search(regex, file)
    # print(gp)
    if gp is not None:
        gopr_key_list.append(file)
    else:
        pass

print("GP KEY LIST: " + str(gopr_key_list))
#expected: ['B_e_1_220918_c1_p1_2.MP4', 'B_e_1_220918_c2_p1_2.MP4', 'B_e_1_220918_c3_p1_2.MP4'...]
#(one file per go pro)


FILE: GX010172_cam4.MP4
FILE: GX010178_cam2.MP4
FILE: GX010178_cam7.MP4
FILE: GX010183_cam6.MP4
FILE: GX010183_cam9.MP4
FILE: GX010196_cam3.MP4
FILE: GX010198_cam8.MP4
FILE: GX010211_cam1.MP4
FILE: GX020172_cam4.MP4
FILE: GX020178_cam2.MP4
FILE: GX020178_cam7.MP4
FILE: GX020183_cam6.MP4
FILE: GX020183_cam9.MP4
FILE: GX020196_cam3.MP4
FILE: GX020198_cam8.MP4
FILE: GX020211_cam1.MP4
FILE: GX030172_cam4.MP4
FILE: GX030178_cam2.MP4
FILE: GX030178_cam7.MP4
FILE: GX030183_cam6.MP4
FILE: GX030183_cam9.MP4
FILE: GX030196_cam3.MP4
FILE: GX030198_cam8.MP4
FILE: GX030211_cam1.MP4
FILE: GX040178_cam2.MP4
FILE: GX040183_cam6.MP4
FILE: GX040183_cam9.MP4
FILE: GX040196_cam3.MP4
FILE: GX040198_cam8.MP4
FILE: GX040211_cam1.MP4
GP KEY LIST: ['GX010172_cam4.MP4', 'GX010178_cam2.MP4', 'GX010178_cam7.MP4', 'GX010183_cam6.MP4', 'GX010183_cam9.MP4', 'GX010196_cam3.MP4', 'GX010198_cam8.MP4', 'GX010211_cam1.MP4']


In [17]:

for file in gopr_key_list:
    filenum = file[4:]
    fstring = f"(GX0\\d{{1}}{filenum})"
    r = re.compile(fstring)
    gplist = list(filter(r.match, file_list))
    gopr_dict.update({file:gplist})

print(gopr_dict)
# expected: {'GX010044_cam9.MP4': ['GX010044_cam9.MP4', 'GX020044_cam9.MP4'],...}
# expected: {'B_e_1_220918_c1_p1_2.MP4': ['B_e_1_220918_c1_p1_2.MP4', 'B_e_1_220918_c1_p2_2.MP4'], ...}


{'GX010172_cam4.MP4': ['GX010172_cam4.MP4', 'GX020172_cam4.MP4', 'GX030172_cam4.MP4'], 'GX010178_cam2.MP4': ['GX010178_cam2.MP4', 'GX020178_cam2.MP4', 'GX030178_cam2.MP4', 'GX040178_cam2.MP4'], 'GX010178_cam7.MP4': ['GX010178_cam7.MP4', 'GX020178_cam7.MP4', 'GX030178_cam7.MP4'], 'GX010183_cam6.MP4': ['GX010183_cam6.MP4', 'GX020183_cam6.MP4', 'GX030183_cam6.MP4', 'GX040183_cam6.MP4'], 'GX010183_cam9.MP4': ['GX010183_cam9.MP4', 'GX020183_cam9.MP4', 'GX030183_cam9.MP4', 'GX040183_cam9.MP4'], 'GX010196_cam3.MP4': ['GX010196_cam3.MP4', 'GX020196_cam3.MP4', 'GX030196_cam3.MP4', 'GX040196_cam3.MP4'], 'GX010198_cam8.MP4': ['GX010198_cam8.MP4', 'GX020198_cam8.MP4', 'GX030198_cam8.MP4', 'GX040198_cam8.MP4'], 'GX010211_cam1.MP4': ['GX010211_cam1.MP4', 'GX020211_cam1.MP4', 'GX030211_cam1.MP4', 'GX040211_cam1.MP4']}


In [22]:

if cameras != "all":
    #gopr_dict = {list(gopr_dict)[cam2list[k]]: gopr_dict[list(gopr_dict)[cam2list[k]]] for k in cameras}
    #find the cameras with cam4 if 4 is in the list with cameras
    gopr_dict = {key: value for key, value in gopr_dict.items() if any(f'_cam{camera}' in key for camera in cameras)}
    print(gopr_dict)

#Read the dictionary and create txt files which contains files which belong together
for key in gopr_dict:
    camera = key[-5]
    print(camera)

    gpr_txt_path = output_path / ('cam' + camera + '.txt')
    print(gpr_txt_path.exists())
    if gpr_txt_path.exists() is True:
        print("PASS on TXT FILE")
        pass
    else:
        gpr_sources_txt = open(gpr_txt_path, 'a')

        gprfile_list = sorted(gopr_dict[key])
        for file in gprfile_list:
            file_stmnt = "file " + '\'' + str(input_path / file) + '\'\n'
            gpr_sources_txt.write(file_stmnt)

        gpr_sources_txt.close()
        
        
    #Join videos which come from the same camera
    mp4_output = output_path / (f'{group_letter}_e{experiment_number}_c{camera}_joined.MP4')
    if mp4_output.exists():
        print("PASS on joined files")
        pass
    else:
        ffmpeg_cmd = [
                                'ffmpeg', '-safe', '0', '-f', 'concat',  '-i',
                                gpr_txt_path, '-c', 'copy',
                            mp4_output
                            ]
        ffmpeg_cmd = [
            'ffmpeg', '-analyzeduration', '100M', '-probesize', '50M', '-safe', '0', '-f', 'concat', '-i',
            gpr_txt_path, '-vf', 'scale=1920:1080', '-c:v', 'libx264', '-crf', '23', '-preset', 'medium',
            '-c:a', 'aac', '-b:a', '192k',
            mp4_output
        ]

        subprocess.call(ffmpeg_cmd, shell=False)



4
True
PASS on TXT FILE


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

2
True
PASS on TXT FILE


[concat @ 0x5c448cc678c0] Could not find codec parameters for stream 2 (Unknown: none): unknown codec
Consider increasing the value for the 'analyzeduration' (100000000) and 'probesize' (50000000) options
Input #0, concat, from '/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/Converted/R1/cam2.txt':
  Duration: N/A, start: 0.000000, bitrate: 45057 kb/s
  Stream #0:0(eng): Video: hevc (Main) (hvc1 / 0x31637668), yuvj420p(pc, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 44817 kb/s, 29.97 fps, 29.97 tbr, 30k tbn, 29.97 tbc
    Metadata:
      creation_time   : 2021-06-04T14:52:20.000000Z
      handler_name    : GoPro H.265
      vendor_id       : [0][0][0][0]
      encoder         : GoPro H.265 encoder
      timecode        : 14:51:26:29
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 189 kb/s
    Metadata:
      creation_time   : 2021-06-04T14:52:20.000000Z
      handler_name    : GoPro AAC  
      vendor_id       : [0][0][0][0]
      timecode       

7
True
PASS on TXT FILE
PASS on joined files
6
True
PASS on TXT FILE


...

[concat @ 0x5735f28898c0] Could not find codec parameters for stream 2 (Unknown: none): unknown codec
Consider increasing the value for the 'analyzeduration' (100000000) and 'probesize' (50000000) options
Input #0, concat, from '/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/Converted/R1/cam3.txt':
  Duration: N/A, start: 0.000000, bitrate: 45008 kb/s
  Stream #0:0(eng): Video: hevc (Main) (hvc1 / 0x31637668), yuvj420p(pc, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 44769 kb/s, 29.97 fps, 29.97 tbr, 30k tbn, 29.97 tbc
    Metadata:
      creation_time   : 2023-06-04T14:49:18.000000Z
      handler_name    : GoPro H.265
      vendor_id       : [0][0][0][0]
      encoder         : GoPro H.265 encoder
      timecode        : 14:48:25:01
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 189 kb/s
    Metadata:
      creation_time   : 2023-06-04T14:49:18.000000Z
      handler_name    : GoPro AAC  
      vendor_id       : [0][0][0][0]
      timecode       

8
True
PASS on TXT FILE


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1
True
PASS on TXT FILE


[concat @ 0x5d4c2d3048c0] Could not find codec parameters for stream 2 (Unknown: none): unknown codec
Consider increasing the value for the 'analyzeduration' (100000000) and 'probesize' (50000000) options
Input #0, concat, from '/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/Converted/R1/cam1.txt':
  Duration: N/A, start: 0.000000, bitrate: 45034 kb/s
  Stream #0:0(eng): Video: hevc (Main) (hvc1 / 0x31637668), yuvj420p(pc, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 44795 kb/s, 29.97 fps, 29.97 tbr, 30k tbn, 29.97 tbc
    Metadata:
      creation_time   : 2022-06-04T14:52:41.000000Z
      handler_name    : GoPro H.265
      vendor_id       : [0][0][0][0]
      encoder         : GoPro H.265 encoder
      timecode        : 14:51:48:07
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 189 kb/s
    Metadata:
      creation_time   : 2022-06-04T14:52:41.000000Z
      handler_name    : GoPro AAC  
      vendor_id       : [0][0][0][0]
      timecode       

In [23]:
#path = "/Users/vogg/Documents/Videos/Lemurs/MadagascarTempVideos/Group_B/Full/"
file_list = sorted(os.listdir(output_path))
file_list = [f for f in file_list if f.endswith("joined.MP4")]
print(file_list)

['R_e9_c1_joined.MP4', 'R_e9_c2_joined.MP4', 'R_e9_c3_joined.MP4', 'R_e9_c4_joined.MP4', 'R_e9_c6_joined.MP4', 'R_e9_c7_joined.MP4', 'R_e9_c8_joined.MP4', 'R_e9_c9_joined.MP4']


In [24]:
## extract audio files from joined videos

for file in file_list:

    if os.path.isfile(output_path / (file[:-4] + ".wav")):
        pass

    else:
        

        #ffmpeg_cmd = [
        #            'ffmpeg', '-i', output_path / file, '-ab', '160k', '-ac', '2',
        #            '-ar', '44100', '-vn', output_path / (file[:-4] + ".wav")
        #            ]
        
        ffmpeg_cmd = [
        'ffmpeg', '-err_detect', 'ignore_err', '-i', str(output_path / file), 
        '-ab', '160k', '-ac', '2', '-ar', '44100', '-vn', str(output_path / (file[:-4] + ".wav"))
        ]

        subprocess.call(ffmpeg_cmd, #was Popen
                            shell=False)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [28]:



anchor_video_idx = [idx for idx, s in enumerate(file_list) if 'c7' in s][0]
print("Anchor video: ", anchor_video_idx)

channels, nChannels, sampleRate, ampWidth, nFrames = extract_audio(str(output_path / (file_list[anchor_video_idx][:-4] + ".wav")), 
                                                                   tStart, tEnd)
samples = convert_to_mono(channels, nChannels, np.int16)


samples = samples[::100]



#plot_audio_samples("Clicker Training", samples, sampleRate, tStart, tEnd)

Anchor video:  5


### Audio matching: Find the time difference

Creates a black (empty) video of the corresponding length.

In [29]:
for idx, item in enumerate(file_list):
    if idx != anchor_video_idx:
        channels2, nChannels2, sampleRate2, ampWidth2, nFrames2 = extract_audio(str(output_path / (file_list[idx][:-4] + ".wav")), tStart, tEnd2)
        samples2 = convert_to_mono(channels2, nChannels2, np.int16)

        samples2 = samples2[::100]
        print("Processing video: ", idx, item)
        print(len(samples2))
        print(sampleRate)

        #abs_diff = np.correlate(samples, samples2)
        abs_diff = get_abs_diff(samples, samples2)

        peaks, properties = find_peaks(-np.array(abs_diff), prominence = 5000, width = (0,40), wlen = 40)
        sorted_prominences = sorted(zip(properties['prominences'], peaks), key=lambda x: x[0], reverse=True)
        max_element, corresponding_element = sorted_prominences[0] #should be 0 normally
        #max_element, corresponding_element = max(zip(properties['prominences'], peaks), key=lambda x: x[0])
        delay_s = round(corresponding_element/(sampleRate/100), 3)
        delay_frames = round(corresponding_element/(sampleRate/100) * 30)

        plt.figure()
        plt.plot(range(len(abs_diff)), abs_diff)
        plt.plot(corresponding_element, np.array(abs_diff)[corresponding_element], "x")
        plt.savefig(path_to_files +"Converted/"+ item[:-11]+'.png')
        plt.close()
        #delay_s = round(np.argmin(abs_diff)/(sampleRate/100), 3)
        #delay_frames = round(np.argmin(abs_diff)/(sampleRate/100) * 30)
        print(str(delay_s) + "s = " + str(delay_frames) + " frames")
        ## Add black frames to beginning of video
        #https://video.stackexchange.com/questions/20717/ffmpeg-add-3-seconds-of-black-to-video-head-and-tail
        ffmpeg_cmd = [
                            'ffmpeg', '-y', '-i', output_path / file_list[idx], '-vf', 'trim=0:'+str(delay_s)+',geq=0:128:128', 
                            '-af', 'atrim=0:'+ str(delay_s) + ',volume=0',
                            '-video_track_timescale', '600', output_path / ("blank" + str(idx) + ".mp4")
                            ]

        subprocess.run(ffmpeg_cmd, shell=False)


Processing video:  0 R_e9_c1_joined.MP4
44100
44100
19.435s = 583 frames


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Processing video:  1 R_e9_c2_joined.MP4
44100
44100
16.399s = 492 frames


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Processing video:  2 R_e9_c3_joined.MP4
44100
44100
35.585s = 1068 frames


...

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Processing video:  6 R_e9_c8_joined.MP4
44100
44100
22.272s = 668 frames


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Processing video:  7 R_e9_c9_joined.MP4
44100
44100
27.882s = 836 frames


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Join the blank videos with the original videos

In [30]:
for idx, item in enumerate(file_list):
    if idx != anchor_video_idx:        
        ffmpeg_cmd = [
                        'ffmpeg', '-y', '-i', output_path / ("blank" + str(idx) + ".mp4"), '-i', output_path / file_list[idx], 
                        '-filter_complex', '[0:v]fps=fps=30[video0];[1:v]fps=fps=30[video1];[video0][0:a:0][video1][1:a:0]concat=n=2:v=1:a=1[outv][outa]', #'[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]', 
                        '-map', "[outv]", '-map', "[outa]",
                        output_path / (item[:-11] + ".mp4")
                            ]


        subprocess.run(ffmpeg_cmd, shell=False)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Convert the anchor video (cam7)

In [31]:
#print(output_path + file_list[anchor_video_idx])

ffmpeg_cmd = [
                        'ffmpeg', '-y','-i', output_path / file_list[anchor_video_idx], 
                        '-filter_complex', '[0:v:0][0:a:0]concat=n=1:v=1:a=1[outv][outa]', 
                        '-map', "[outv]", '-map', "[outa]",
                        output_path / (file_list[anchor_video_idx][:-11] + ".mp4")
                        ]


subprocess.run(ffmpeg_cmd, shell=False)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

CompletedProcess(args=['ffmpeg', '-y', '-i', PosixPath('/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/Converted/R1/R_e9_c7_joined.MP4'), '-filter_complex', '[0:v:0][0:a:0]concat=n=1:v=1:a=1[outv][outa]', '-map', '[outv]', '-map', '[outa]', PosixPath('/usr/users/vogg/sfb1528s3/B06/2024 march-june/reminder lift/Converted/R1/R_e9_c7.mp4')], returncode=0)

In [32]:
## Remove all files that are not used anymore

for fname in os.listdir(output_path):
    if "cam" in fname:
        os.remove(os.path.join(output_path, fname))

for fname in os.listdir(output_path):
    if fname.startswith("blank") or ("joined" in fname):
        os.remove(os.path.join(output_path, fname))
              

     